In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import tensorflow_addons as tfa
import math

import datetime
from datetime import datetime
from packaging import version

from keras import regularizers

from hyperas import optim
from hyperopt import Trials, tpe

#from transformers import TFViTModel

import cv2
from keras_preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

In [2]:
num_classes = 2
input_shape = (256, 256, 3)

learning_rate = 0.01
weight_decay = 0.0001
batch_size = 32
num_epochs = 30
image_size = 224  # We'll resize input images to this size
patch_size = 16  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 128 # you can change this
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 16 # you can change this (ex. make it 6)
mlp_head_units = [256, 128]  # Size of the dense layers of the final classifier

hyperparameters = {'batch_size': 128,
                   'random_flip': 'horizontal_and_vertical',
                   'random_rotation': (0.2),
                   'dropout': 0.5,
                   'L2': 0.3,
                   'base_LR': 0.001,
                   'initial_epochs': 20,
                   'fine_tune_epochs': 50,
                   'frozen_layer': 72}

SEED = 42
keras.utils.set_random_seed(SEED)

path = '/Users/alexk/Documents/GitHub/Deep_Learning_for_Binary_Classification_of_Infectious_Keratitis/data'

In [3]:
from keras.applications.efficientnet import EfficientNetB0
model = EfficientNetB0(weights='imagenet')

Metal device set to: Apple M1 Max


In [4]:
model = EfficientNetB0(include_top=False, weights='imagenet')

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset='training',
    seed=42,
    batch_size=batch_size,
    shuffle=True,
    label_mode='categorical',
    image_size=(image_size, image_size),
    class_names=['Bacterial', 'Fungal'])

val_ds = keras.preprocessing.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset='validation',
    seed=42,
    label_mode='categorical',
    shuffle=True,
    batch_size=batch_size,
    image_size=(image_size, image_size),
    class_names=['Bacterial', 'Fungal'])

Found 671 files belonging to 2 classes.
Using 537 files for training.
Found 671 files belonging to 2 classes.
Using 134 files for validation.


In [6]:
test_path = "/Users/alexk/Documents/GitHub/Deep_Learning_for_Binary_Classification_of_Infectious_Keratitis/data"
x_test = []
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

for folder in listdir_nohidden(test_path):

    sub_path=test_path+"/"+folder

    for img in listdir_nohidden(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

In [7]:
#predicted_classes = np.argmax(mlpmixer_classifier.predict(X, axis = 1))
test_datagen = ImageDataGenerator(rescale = 1./255)

test_x=np.array(x_test)

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

test_y=test_set.classes

print(test_y.shape)

Found 671 images belonging to 2 classes.
(671,)


In [8]:
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    img_augmentation = keras.models.Sequential(
        [
            layers.RandomRotation(factor=0.15),
            layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
            layers.RandomFlip(),
            layers.RandomContrast(factor=0.1),
        ],
        name="img_augmentation",
    )

In [9]:
def input_preprocess(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label


#train_ds = train_ds.map(
#    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
#)
#train_ds = train_ds.batch(batch_size=batch_size, drop_remainder=True)
#train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

#val_ds = val_ds.map(input_preprocess)
#val_ds = val_ds.batch(batch_size=batch_size, drop_remainder=True)

In [10]:
print(train_ds)

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>


In [11]:
def build_model(num_classes):
    inputs = layers.Input(shape=(image_size, image_size, 3))
    with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
        x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", "AUC"]
    )
    return model

In [12]:
model = build_model(num_classes=num_classes)
model.summary()
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    hist = model.fit(train_ds, epochs=num_epochs, validation_data=val_ds, verbose=2, callbacks=[early_stopping])

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 img_augmentation (Sequential)  (None, 224, 224, 3)  0           ['input_3[0][0]']                
                                                                                                  
 rescaling_2 (Rescaling)        (None, 224, 224, 3)  0           ['img_augmentation[0][0]']       
                                                                                                  
 normalization_2 (Normalization  (None, 224, 224, 3)  7          ['rescaling_2[0][0]'] 

2023-04-17 10:31:55.752831: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


KeyboardInterrupt: 

In [38]:
resnet50 = keras.applications.resnet50

conv_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

for layer in conv_model.layers:
    layer.trainable = False
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
        x = img_augmentation(conv_model.output)
x = keras.layers.Flatten()(conv_model.output)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
x = keras.layers.Dense(100, activation='relu')(x)
predictions = keras.layers.Dense(2, activation='softmax')(x)
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)
full_model.summary()

res_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

full_model.compile(
    optimizer=res_optimizer, loss="categorical_crossentropy", metrics=["accuracy", "AUC"]
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_16[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [39]:
res_early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
with tf.device('/job:localhost/replica:0/task:0/device:CPU:0'):
    history = full_model.fit(train_ds, epochs=num_epochs, validation_data=val_ds, verbose=2, callbacks=[res_early_stopping])

Epoch 1/30
17/17 - 31s - loss: 108.3388 - accuracy: 0.5363 - auc: 0.5495 - val_loss: 8.4607 - val_accuracy: 0.6343 - val_auc: 0.6444 - 31s/epoch - 2s/step
Epoch 2/30
17/17 - 29s - loss: 28.4586 - accuracy: 0.5549 - auc: 0.5567 - val_loss: 3.7111 - val_accuracy: 0.5672 - val_auc: 0.5847 - 29s/epoch - 2s/step
Epoch 3/30
17/17 - 28s - loss: 1.0768 - accuracy: 0.8082 - auc: 0.8633 - val_loss: 1.0000 - val_accuracy: 0.6343 - val_auc: 0.7323 - 28s/epoch - 2s/step
Epoch 4/30
17/17 - 29s - loss: 0.5146 - accuracy: 0.8454 - auc: 0.9278 - val_loss: 1.0553 - val_accuracy: 0.7313 - val_auc: 0.8248 - 29s/epoch - 2s/step
Epoch 5/30
17/17 - 31s - loss: 0.1487 - accuracy: 0.9497 - auc: 0.9855 - val_loss: 0.8750 - val_accuracy: 0.8060 - val_auc: 0.8748 - 31s/epoch - 2s/step
Epoch 6/30
17/17 - 28s - loss: 0.0884 - accuracy: 0.9683 - auc: 0.9929 - val_loss: 2.0940 - val_accuracy: 0.7090 - val_auc: 0.7879 - 28s/epoch - 2s/step
Epoch 7/30
17/17 - 27s - loss: 0.0717 - accuracy: 0.9795 - auc: 0.9947 - val_lo

In [30]:
mobilenet_predicted_classes = np.argmax(model.predict(test_x), axis=1)
mobilenet_cm = confusion_matrix(test_y, mobilenet_predicted_classes)
print(mobilenet_cm)

mobilenet_TN = mobilenet_cm[0][0]
mobilenet_FN = mobilenet_cm[0][1]
mobilenet_FP = mobilenet_cm[1][0]
mobilenet_TP = mobilenet_cm[1][1]
mobilenet_sensitivity = mobilenet_TP / (mobilenet_TP + mobilenet_FN)
mobilenet_specificity = mobilenet_TN / (mobilenet_TN + mobilenet_FP)

mobilenet_accuracy = (mobilenet_TP + mobilenet_TN) / 671

print(mobilenet_sensitivity)
print(mobilenet_specificity)
print(mobilenet_accuracy)

[[ 79 152]
 [ 72 368]]
0.7076923076923077
0.5231788079470199
0.6661698956780924


In [40]:
resnet_predicted_classes = np.argmax(full_model.predict(test_x), axis=1)
resnet_cm = confusion_matrix(test_y, resnet_predicted_classes)
print(resnet_cm)

resnet_TN = resnet_cm[0][0]
resnet_FN = resnet_cm[0][1]
resnet_FP = resnet_cm[1][0]
resnet_TP = resnet_cm[1][1]
resnet_sensitivity = resnet_TP / (resnet_TP + resnet_FN)
resnet_specificity = resnet_TN / (resnet_TN + resnet_FP)

resnet_accuracy = (resnet_TP + resnet_TN) / 671

print(resnet_sensitivity)
print(resnet_specificity)
print(resnet_accuracy)

[[209  22]
 [ 65 375]]
0.9445843828715366
0.7627737226277372
0.8703427719821163
